In [ ]:
##################################################################################################################
#
# GuitarML SmartPedal Colab Capture Instructions
#
#  Step 1. Click "Runtime" in the Colab top menu bar, then "Change Runtime Type"
#          and select "GPU" and save. You have a limited number of consecutive GPU hours with
#          the free version of Colab, but this will reset in a day or so. 
#
#  Step 2. Upload your recorded output from your device by dragging and dropping the "out.wav"
#          file to the file browser in the left hand window in Colab. Ensure that this file is named
#          "out.wav".
#
#  Step 3. Run the SETUP ENVIRONMENT section by clicking the run arrow on the top left of the box.
#          Should take a few minutes. Scroll down to next step when finished.
#
#  Step 4. Choose one (and only one) of the next 3 options as applicable to your captured device. This
#          will begin the model training process. It should take about 12 minutes with GPU runtime selected.
#          You may stop and resume this block if needed, the process will continue where it left off.
#          
#  Step 5. Run this section to generate a Smart Pedal compatible model file. Download from the 
#          left hand file browser by right clicking the "newSmartPedal.json" filename in the left hand menu and 
#          selecting "Download". May need to refresh the filebrowser by clicking the refresh folder icon.
#
#  Step 6. Optionally, generate a plot of the signals to visually see how close the model is 
#          to the actual device. 
#
#  Note: Recommended to "Disconect and delete runtime" from the "Runtime" menu and "Reconnect" to reset the environment
#     when training additional models. Make sure to download your trained model before disconnecting.
#
#  Note: To continue to refine the model, you may run the same Step 4 (a,b,c) after increasing the "--max_epochs" field value.
#     for example, in Step 4c., change "--max_epochs=1300" to "--max_epochs=1400" to run and additional 100 epochs. 
#     Then re-run Step 5 to generate a new Smart Pedal compatible model. 
#
##################################################################################################################


In [ ]:
# STEP 3 : SETUP ENVIRONMENT
# Run this block to setup environment. May take a few minutes.

!git clone https://github.com/GuitarML/PedalNetRT.git
%cd PedalNetRT
!git checkout smartpedal-capture
!pip install -r requirements-colab.txt

In [ ]:
# STEP 4a. Run this to start capture of CLEAN AMP or COMPRESSOR
model="models/amp_clean/amp_clean_ht40.ckpt"
!python train.py data/SmartPedal_Capture.wav ../out.wav --resume --max_epochs=2300 --model=$model --cpu

In [ ]:
# STEP 4b. Run this to start capture of MEDIUM to HIGH GAIN AMP
model="models/amp_gain/amp_gain_ht40.ckpt"
!python train.py ../SmartPedal_Capture.wav ../out.wav --resume --max_epochs=1300 --model=$model

In [ ]:
# STEP 4c.  Run this to start capture of OVERDRIVE, DISTORTION, or BOOST PEDAL
model="models/pedal_overdrive/pedal_overdrive_ts9.ckpt"
!python train.py data/SmartPedal_Capture.wav ../out.wav --resume --max_epochs=1300 --model=$model

In [4]:
# STEP 5. After Above Capture Process is Completed, Run this to generate a Smart Pedal compatible model file.
#         The model file will be located in the top level folder with the name "newSmartPedalModel.json"
#         May need to refresh the left hand file browser by clicking the refresh folder icon.
#         Right click the file and choose download, then rename as appropriate.

!python export.py --model=$model
%cp $(dirname $model)"/"$(basename $model ".ckpt")".json" ../
%mv ../$(basename $model ".ckpt")".json" ../newSmartPedalModel.json

In [ ]:
# STEP 6. (Optional) After Above Capture Process is Completed, Run this to generate data graphs of the device model
#         The PNG image will be located in the top level folder with the name "detail_signal_comparison_e2s...".
#         May need to refresh the left hand file browser by clicking the refresh folder icon.

# Double click the PNG file to view the graphs. These graphs show approximately 100 milliseconds of audio signal. 
# The top plot shows the input signal. The middle plot shows the target device signal vs the signal 
#      predicted by the new model. The goal is for these signals to be as close as possible.

!python test.py --model=$model
!python plot.py --model=$model
%cp $(dirname $model)"/"detail* ../

In [ ]:
## ADDITIONAL TECHNICAL NOTES ##

# 1. Training will go through 300 epochs, beginning with a pre-trained starting point model.
#    The "max_epochs" field values are much higher due to the previous training of the model. 
#    For example, the pre-trained high gain amp model was trained for 1000 epochs, so the max_epochs are 
#    set to 1300 in the train.py command to conduct 300 additional epochs of training.
#    The Clean Amp pre-trained model was trained for 2000 epochs so it is set to 2300 by default. 
#    This can be adjusted to more or less by changing the value in the "max_epochs" field.

# 2. If you run out of GPU runtime usage in Colab, add "--cpu" flag to the "train.py" command to
#    train on the cpu. This will be much slower.
# 
#    Example:
#       change this:
#                     !python train.py data/SmartPedal_Capture.wav ../out.wav --resume --max_epochs=2300 --model=$model
#       to this:
#                     !python train.py data/SmartPedal_Capture.wav ../out.wav --resume --max_epochs=2300 --model=$model --cpu
#
# 3. This catpure method is limited to non-time based effects (no Delay, Reverb, Flange, Phaser, etc.) 
#      It is intended for Amplifiers, Distortion/Overdrive/Boost and Compression(results on compressors vary)
# 